In [11]:
import sys
from pathlib import Path

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

In [12]:
%reload_ext autoreload
%autoreload 2

(opcional) Este código permite conocer lo que hay en hopsworks

In [13]:
# Conocer que datos hay en el feature store
import hopsworks
import pandas as pd
import config

FG_NAME = "times_series_bolleria_feature_group"
FV_NAME = "times_series_bolleria_feature_view"
FV_VERSION = 1

# Conectar con el proyecto de Hopsworks y al feature store
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)


# Obtener el feature store
feature_store = project.get_feature_store()

# Listar los feature groups
feature_groups = feature_store.get_feature_groups(name=FG_NAME)
print(f"Feature groups encontrados: {len(feature_groups)}")
for fg in feature_groups:
    print(f" - {fg.name}")



2025-08-21 17:54:26,729 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-21 17:54:26,733 INFO: Initializing external client
2025-08-21 17:54:26,733 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-08-21 17:54:26,733 INFO: Initializing external client
2025-08-21 17:54:26,733 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-21 17:54:28,011 INFO: Python Engine initialized.
2025-08-21 17:54:28,011 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
Feature groups encontrados: 1
 - times_series_bolleria_feature_group
Feature groups encontrados: 1
 - times_series_bolleria_feature_group


In [14]:
# Para cada FG, saca numero de filas y rango de timestamp
for fg in feature_groups:
    print(f"Feature group: {fg.name}")
    df = fg.read()
    print(f" - Número de filas: {len(df)}")
    if not df.empty:
        print(f" - Rango de week_start: {df['week_start'].min()} a {df['week_start'].max()}")
    else:
        print(" - No hay datos en este feature group.")

Feature group: times_series_bolleria_feature_group
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00


In [15]:
# Listas los features views
feature_views = feature_store.get_feature_views(name=FV_NAME)
# Filtrar por versión
feature_views = [fv for fv in feature_views if fv.version == FV_VERSION]
print(f"Feature views encontrados: {len(feature_views)}")
for fv in feature_views:
    print(f" - {fv.name}")
    print(f" - Version: {fv.version}")

# Para cada feature view saca el número de filas y rango de timestamp
batch_data = []
for fv in feature_views:
    df = fv.get_batch_data()
    batch_data.append(df)

# Imprimir información sobre los datos de cada feature view y las columnas
for i, df in enumerate(batch_data):
    print(f"Feature view {feature_views[i].name}:")
    print(f" - Número de filas: {len(df)}")
    if not df.empty:
        print(f" - Rango de week_start: {df['week_start'].min()} a {df['week_start'].max()}")
        print(f" - Columnas: {list(df.columns)}")
    else:
        print(" - No hay datos en este feature view.")

Feature views encontrados: 1
 - times_series_bolleria_feature_view
 - Version: 1
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 
Feature view times_series_bolleria_feature_view:
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00
 - Columnas: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 
Feature view times_series_bolleria_feature_view:
 - Número de filas: 133
 - Rango de week_start: 2023-01-02 00:00:00+00:00 a 2025-08-11 00:00:00+00:00
 - Columnas: ['familia', 'base_imponible', 'is_summer_peak', 'is_easter', 'week_start']


# Conexión al proyecto, feature store y model registry en hopsworks

In [16]:
import hopsworks

# Conexión a hopsworks y proyecto   
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)
feature_store = project.get_feature_store()

# Conexión al model registry
model_registry = project.get_model_registry()

print("Conectado a proyecto:", project.name)
print("Conectado al feature store:", feature_store.name)
print("Conectado al model registry:", model_registry)


2025-08-21 17:54:43,723 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-21 17:54:43,727 INFO: Initializing external client
2025-08-21 17:54:43,727 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-08-21 17:54:43,727 INFO: Initializing external client
2025-08-21 17:54:43,727 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-21 17:54:44,869 INFO: Python Engine initialized.
2025-08-21 17:54:44,869 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
Conectado a proyecto: fleca_mlops
Conectado al feature store: fleca_mlops_featurestore
Conectado al model registry: ModelRegistry(project: 'fleca_mlops')
Conectado a proyecto: fleca_mlops
Conectado al feature store: fleca_mlops_featurestore
Conectado al model registry: ModelRegistry(project: 'fleca_mlops')


# Leemos Batch Score del Feature View

In [17]:
from datetime import timedelta
import pandas as pd

# Lectura batch cruda de feature views
feature_views= feature_store.get_feature_view(name=FV_NAME, version=FV_VERSION)
ts_df = feature_views.get_batch_data(
    start_date=None,
    end_date=None,
    features=None,
    )

# Ordenar por la columna de fecha antes de visualizar
ts_df = ts_df.sort_values('week_start').reset_index(drop=True)
ts_df.head(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.91s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.91s) 


,familia,base_imponible,is_summer_peak,is_easter,week_start
0,BOLLERIA,825.11,0,0,2023-01-02 00:00:00+00:00
1,BOLLERIA,658.40,0,0,2023-01-09 00:00:00+00:00
2,BOLLERIA,741.40,0,0,2023-01-16 00:00:00+00:00
3,BOLLERIA,653.64,0,0,2023-01-23 00:00:00+00:00
4,BOLLERIA,680.46,0,0,2023-01-30 00:00:00+00:00


# Añadimos features (lags, variables exógenas) y target

In [18]:
from src.model import transformar_features_target

# Procesar datos usando la función mejorada que acepta tuplas directamente
ts_df = transformar_features_target(
    ts_df,
    lags_list=[1, 2, 3, 52], 
    columna_target='base_imponible',
    cols_exogenas=['is_easter', 'is_summer_peak'],
    periodos_adelante=1,
    eliminar_nulos=True,
    return_format='dataframe'  # Obtenemos un único DataFrame con features y target
)

# Resetear el índice
ts_df = ts_df.reset_index(drop=True)

# Mostrar las primeras filas del DataFrame transformado
ts_df.head(5)

2025-08-21 17:54:57,566 INFO: Usando DataFrame de entrada: (133, 5)
2025-08-21 17:54:57,569 INFO: Retornando DataFrame combinado: (80, 8)
2025-08-21 17:54:57,569 INFO: Retornando DataFrame combinado: (80, 8)


,base_imponible_lag1,base_imponible_lag2,base_imponible_lag3,base_imponible_lag52,is_easter,is_summer_peak,week_start,target
0,572.51,534.79,563.18,825.11,0,0,2024-01-15 00:00:00+00:00,680.30
1,597.65,572.51,534.79,658.40,0,0,2024-01-22 00:00:00+00:00,603.99
2,680.30,597.65,572.51,741.40,0,0,2024-01-29 00:00:00+00:00,600.14
3,603.99,680.30,597.65,653.64,0,0,2024-02-05 00:00:00+00:00,689.32
4,600.14,603.99,680.30,680.46,0,0,2024-02-12 00:00:00+00:00,627.76


# Cargamos el modelo

In [21]:
import joblib
# Cargar el modelo del model registry
## se obtiene del config

MODEL_NAME = "fleca_bolleria_predictor_next_week"
MODEL_VERSION = 1
MODEL_FILE ='xgboost_hopsworks.pkl'


model_registry = project.get_model_registry()
model = model_registry.get_model(name=MODEL_NAME, version=MODEL_VERSION)    
from pathlib import Path

model_dir = model.download()
model = joblib.load(Path(model_dir) / MODEL_FILE)

print(f'Modelo {MODEL_NAME} V{MODEL_VERSION} cargado correctamente')

Downloading: 0.000%|          | 0/78699 elapsed<00:00 remaining<?

Modelo fleca_bolleria_predictor_next_week V1 cargado correctamente


In [22]:
# Verificar qué columnas tenemos después de transformar_features_target
print("Columnas después de transformar_features_target:")
print(ts_df.columns.tolist())

print("\nFeatures que espera el modelo:")
if 'model' in globals():
    print(model.feature_names_in_)
else:
    print("❌ El modelo no está definido. Por favor, ejecuta la celda que carga el modelo antes de continuar.")

print("\nComparación:")
for feature in model.feature_names_in_:
    if feature in ts_df.columns:
        print(f"✅ {feature}: EXISTE")
    else:
        print(f"❌ {feature}: NO EXISTE")
        
# Verificar si existen las versiones con 'target_lag'
target_lags = [col for col in ts_df.columns if 'target_lag' in col]
print(f"\nColumnas target_lag encontradas: {target_lags}")

Columnas después de transformar_features_target:
['base_imponible_lag1', 'base_imponible_lag2', 'base_imponible_lag3', 'base_imponible_lag52', 'is_easter', 'is_summer_peak', 'week_start', 'target']

Features que espera el modelo:
['base_imponible_lag1' 'base_imponible_lag2' 'base_imponible_lag3'
 'base_imponible_lag52' 'is_easter' 'is_summer_peak']

Comparación:
✅ base_imponible_lag1: EXISTE
✅ base_imponible_lag2: EXISTE
✅ base_imponible_lag3: EXISTE
✅ base_imponible_lag52: EXISTE
✅ is_easter: EXISTE
✅ is_summer_peak: EXISTE

Columnas target_lag encontradas: []


In [23]:
model.feature_names_in_


array(['base_imponible_lag1', 'base_imponible_lag2',
       'base_imponible_lag3', 'base_imponible_lag52', 'is_easter',
       'is_summer_peak'], dtype='<U20')

In [24]:
# Limpiar columnas innecesarias
features = ts_df.drop(columns=['week_start'])

# Hacemos predicciones

In [25]:
# Predicción directa para la próxima semana (sin autoregresión)
from datetime import timedelta

print("=== PREDICCIÓN DIRECTA DE LA PRÓXIMA SEMANA ===")

# Tomar la última fila de datos históricos
ultimo_lunes = ts_df['week_start'].max()
last_row = ts_df.iloc[-1].copy()
feature_names = model.feature_names_in_

# Calcular la fecha de la próxima semana
fecha_siguiente = ultimo_lunes + timedelta(days=7)
last_row['week_start'] = fecha_siguiente

# Preparar los features para el modelo
X = last_row[feature_names].values.reshape(1, -1)
prediccion = model.predict(X)[0]

print(f"Fecha predicha: {fecha_siguiente.date()}")
print(f"Predicción base_imponible: {prediccion:.2f}")

# Mostrar resultado en DataFrame
resultado = pd.DataFrame({
    'week_start': [fecha_siguiente],
    'predicted_base_imponible': [prediccion]
})
print(resultado)


=== PREDICCIÓN DIRECTA DE LA PRÓXIMA SEMANA ===
Fecha predicha: 2025-08-11
Predicción base_imponible: 1310.64
                 week_start  predicted_base_imponible
0 2025-08-11 00:00:00+00:00               1310.636108


## Inferencia regresiva (opcional)

In [150]:
import numpy as np
from datetime import timedelta

print("=== PIPELINE DE INFERENCIA AUTOREGRESIVA ===")

# 1. Preparar datos base
ultimo_lunes = ts_df['week_start'].max()
last_row = ts_df.iloc[-1].copy()
feature_names = model.feature_names_in_

print(f"Última fecha histórica: {ultimo_lunes}")
print("Valores iniciales de lags:")
for lag in [1, 2, 3, 52]:
    col_name = f'base_imponible_lag{lag}'
    print(f"  {col_name}: {last_row[col_name]:.2f}")

# 2. Pipeline de predicción autoregresiva
predicciones_anteriores = []
features_pred = []

for semana in range(1, 5):  # 4 semanas futuras
    fecha = ultimo_lunes + timedelta(days=7 * semana)
    nueva_fila = last_row.copy()
    nueva_fila['week_start'] = fecha
    
    print(f"\n--- Semana {semana} ({fecha.date()}) ---")
    
    # Actualizar lags con predicciones anteriores
    for lag in [1, 2, 3, 52]:
        col_name = f'base_imponible_lag{lag}'
        valor_anterior = nueva_fila[col_name]
        
        if lag <= len(predicciones_anteriores):
            # Usar predicción anterior como lag
            nuevo_valor = predicciones_anteriores[-lag]
            nueva_fila[col_name] = nuevo_valor
            print(f"  {col_name}: {valor_anterior:.2f} -> {nuevo_valor:.2f}")
        else:
            # Mantener valor histórico
            print(f"  {col_name}: {valor_anterior:.2f} (histórico)")
    
    # Hacer predicción
    X = nueva_fila[feature_names].values.reshape(1, -1)
    prediccion = model.predict(X)[0]
    print(f"  Predicción: {prediccion:.2f}")
    
    # Guardar para siguiente iteración
    predicciones_anteriores.append(prediccion)
    nueva_fila['predicted_base_imponible'] = prediccion
    features_pred.append(nueva_fila.copy())

# 3. Crear DataFrame de resultados
results = pd.DataFrame(features_pred)[['week_start', 'predicted_base_imponible']]
fecha_limite = pd.Timestamp('2025-08-11 00:00:00+00:00')
results = results[results['week_start'] > fecha_limite].reset_index(drop=True)

print("\n=== RESULTADOS FINALES ===")
print(results)

=== PIPELINE DE INFERENCIA AUTOREGRESIVA ===
Última fecha histórica: 2025-07-28 00:00:00+00:00
Valores iniciales de lags:
  base_imponible_lag1: 1408.04
  base_imponible_lag2: 1318.98
  base_imponible_lag3: 1299.57
  base_imponible_lag52: 1714.05

--- Semana 1 (2025-08-04) ---
  base_imponible_lag1: 1408.04 (histórico)
  base_imponible_lag2: 1318.98 (histórico)
  base_imponible_lag3: 1299.57 (histórico)
  base_imponible_lag52: 1714.05 (histórico)
  Predicción: 1310.64

--- Semana 2 (2025-08-11) ---
  base_imponible_lag1: 1408.04 -> 1310.64
  base_imponible_lag2: 1318.98 (histórico)
  base_imponible_lag3: 1299.57 (histórico)
  base_imponible_lag52: 1714.05 (histórico)
  Predicción: 1329.65

--- Semana 3 (2025-08-18) ---
  base_imponible_lag1: 1408.04 -> 1329.65
  base_imponible_lag2: 1318.98 -> 1310.64
  base_imponible_lag3: 1299.57 (histórico)
  base_imponible_lag52: 1714.05 (histórico)
  Predicción: 1329.65

--- Semana 4 (2025-08-25) ---
  base_imponible_lag1: 1408.04 -> 1329.65
  bas